In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
from os.path import join as oj
import analyze_utils
import matplotlib.pyplot as plt
import iprompt
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
tqdm.pandas()
keys = ['model_cls', 'task_name', 'checkpoint', 'seed']
keys_extra = ['n_shots', 'num_learned_tokens']
keys_out = ['reciprocal_rank', 'prefixes', 'iprompt_preprefix_str']

save_dir = '/home/chansingh/mntv1/iprompt_revision_xmas/'
# out_dir = '/home/chansingh/iprompt/results/human_eval/'
out_dir = '/home/chansingh/interpretable-autoprompting/results/human_eval/'
os.makedirs(out_dir, exist_ok=True)

In [2]:
# d requires running 09_prompts_view_tables first
d = pd.read_pickle(oj(save_dir, 'd.pkl'))
d = d[d.checkpoint == 'EleutherAI/gpt-j-6B']
d = d[~d.task_name.str.startswith('d3_')]
d = d[['task_name', 'prefixes', 'model_cls', 'reciprocal_rank', 'gt_prompt', 'task_collection']]
d = (
    d
    .sort_values(by=['reciprocal_rank'], ascending=False)
    .groupby(by=['model_cls', 'task_collection', 'task_name'])
    .first()
)
df = d.sort_values('task_name')[['gt_prompt', 'prefixes']].reset_index()
df = df.rename(columns={'gt_prompt': 'Groundtruth', 'prefixes': 'Generation'})

In [3]:
pd.options.display.max_colwidth = None
with pd.option_context('display.max_rows', None):
    display(df)

,model_cls,task_collection,task_name,Groundtruth,Generation
0,autoprompt,Induction,active_to_passive,output the passive form of the input,"Q""""the theWords sentences"
1,iprompt,Induction,active_to_passive,output the passive form of the input,Choose a pronoun for each sentence
2,suff,Induction,active_to_passive,output the passive form of the input,Write a program to get through
3,suff,Math,add_two,Return the sum of the inputs.,n>2 m1
4,iprompt,Math,add_two,Return the sum of the inputs.,Create a function named `sum
5,autoprompt,Math,add_two,Return the sum of the inputs.,>:Returns Adding togetherFont accomplish
6,iprompt,Induction,antonyms,output the opposite of,What is the word whose opposite
7,autoprompt,Induction,antonyms,output the opposite of,prevailing '(the ratios emph gle
8,suff,Induction,antonyms,output the opposite of,The code to ascend. You
9,suff,Induction,cause_and_effect,"Each input consists of two sentences, where one is the cause and the other is the outcome. Write The cause sentence",Your code must be between 1


In [4]:
# df.to_csv(oj(out_dir, 'human_eval_with_labels.csv'))
# df['Score (1-5)'] = ''
# df[['Groundtruth', 'Generation', 'Score (1-5)']].to_csv(oj(out_dir, 'human_eval_no_labels.csv'), index=False)
# TASK = "You are given a groundtruth description along with a generated one. On a scale of 1 (worst) to 5 (best), how interpretable and accurate is the generated description?"

# Analyze results

In [14]:
hums = []
for i in [1, 2, 3, 4]:
    df[f'subj{i}'] = pd.read_csv(oj(out_dir, f'human_eval_subj{i}.csv'))['Score (1-5)']
df['human_score_mean'] = df[['subj1', 'subj2']].mean(axis=1)

In [15]:
df.head()

,model_cls,task_collection,task_name,Groundtruth,Generation,subj1,subj2,human_score_mean,subj3,subj4
0,autoprompt,Induction,active_to_passive,output the passive form of the input,"Q""""the theWords sentences",1,1,1.0,1,1.0
1,iprompt,Induction,active_to_passive,output the passive form of the input,Choose a pronoun for each sentence,1,1,1.0,2,2.0
2,suff,Induction,active_to_passive,output the passive form of the input,Write a program to get through,1,2,1.5,1,1.0
3,suff,Math,add_two,Return the sum of the inputs.,n>2 m1,1,1,1.0,1,1.0
4,iprompt,Math,add_two,Return the sum of the inputs.,Create a function named `sum,4,3,3.5,3,3.0


In [16]:
display((df.groupby(['model_cls', 'task_collection']).mean()/5).round(2))

/tmp/ipykernel_1440057/280787490.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  display((df.groupby(['model_cls', 'task_collection']).mean()/5).round(2))


subj1  subj2  human_score_mean  subj3  subj4
model_cls  task_collection                                              
autoprompt ANLI              0.20   0.22              0.21   0.20   0.20
           Induction         0.22   0.21              0.21   0.20   0.22
           Math              0.26   0.24              0.25   0.20   0.20
iprompt    ANLI              0.62   0.50              0.56   0.46   0.62
           Induction         0.49   0.35              0.42   0.38   0.49
           Math              0.68   0.52              0.60   0.64   0.74
suff       ANLI              0.24   0.26              0.25   0.20   0.28
           Induction         0.22   0.23              0.23   0.22   0.26
           Math              0.20   0.20              0.20   0.24   0.24